<a href="https://colab.research.google.com/github/MehrdadDastouri/book_recommendation_system/blob/main/book_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the dataset
# You can download a book dataset from: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
books = pd.read_csv("books.csv")  # Books metadata
ratings = pd.read_csv("ratings.csv")  # User ratings

# Preview datasets
print("Books Dataset:")
print(books.head())
print("\nRatings Dataset:")
print(ratings.head())

# Merge books and ratings datasets
data = ratings.merge(books, on="book_id")
print("\nMerged Dataset:")
print(data.head())

# Create a pivot table (Users x Books matrix)
user_book_matrix = data.pivot_table(index="user_id", columns="title", values="rating")

# Fill NaN values with 0
user_book_matrix.fillna(0, inplace=True)

# Standardize the data (optional: for cosine similarity)
scaler = StandardScaler()
user_book_matrix_scaled = scaler.fit_transform(user_book_matrix)

# Compute cosine similarity between books
book_similarity = cosine_similarity(user_book_matrix_scaled.T)  # Transpose for book-based similarity
book_similarity_df = pd.DataFrame(book_similarity, index=user_book_matrix.columns, columns=user_book_matrix.columns)

# Function to recommend books based on a given book
def recommend_books(book_title, similarity_matrix, top_n=5):
    """
    Recommends books similar to the given book title.

    Args:
    - book_title (str): Title of the book to base recommendations on.
    - similarity_matrix (DataFrame): Cosine similarity matrix of books.
    - top_n (int): Number of top recommendations to return.

    Returns:
    - list: Titles of recommended books.
    """
    if book_title not in similarity_matrix.index:
        print(f"Book '{book_title}' not found in the dataset.")
        return []
    scores = similarity_matrix[book_title].sort_values(ascending=False)[1:top_n+1]
    return scores.index.tolist()

# Example: Recommend books similar to a specific book
book_to_recommend = "The Catcher in the Rye"
recommendations = recommend_books(book_to_recommend, book_similarity_df, top_n=5)

print(f"\nBooks similar to '{book_to_recommend}':")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")